In [8]:
import tkinter as tk
from tkinter import messagebox

class TicTacToe:
    def __init__(self, master):
        self.master = master
        self.master.title("Tic-Tac-Toe Game")
        self.master.geometry("400x400")
        self.board = ['' for _ in range(9)]
        self.player = "X"
        self.mode = None
        self.buttons = []
        self.x_score = 0
        self.o_score = 0
        self.draw_score = 0
        self.create_widgets()

    def create_widgets(self):
        # Creating mode frame
        mode_frame = tk.Frame(self.master)
        mode_frame.pack()

        # Creating radio buttons for mode selection
        self.mode_var = tk.StringVar(value="pvp")
        tk.Radiobutton(mode_frame, text="Player vs Player", variable=self.mode_var, value="pvp", command=self.reset_game).pack(side=tk.LEFT)
        tk.Radiobutton(mode_frame, text="Player vs AI", variable=self.mode_var, value="pvai", command=self.reset_game).pack(side=tk.LEFT)

        # Creating grid frame
        grid_frame = tk.Frame(self.master)
        grid_frame.pack()

        # Creating buttons for game grid
        for i in range(9):
            button = tk.Button(grid_frame, text="", width=10, height=3, command=lambda i=i: self.on_button_click(i))
            button.grid(row=i//3, column=i%3)
            self.buttons.append(button)

        # Creating status label
        self.status_label = tk.Label(self.master, text="Player X's turn", font=("Helvetica", 14))
        self.status_label.pack()

        # Creating score frame
        score_frame = tk.Frame(self.master)
        score_frame.pack()

        # Creating score label
        self.score_label = tk.Label(score_frame, text="X: 0 | O: 0 | Draw: 0", font=("Helvetica", 12))
        self.score_label.pack()

        # Creating reset button
        reset_button = tk.Button(self.master, text="Restart", command=self.reset_game)
        reset_button.pack()

        # Creating exit button
        exit_button = tk.Button(self.master, text="Exit", command=self.master.quit)
        exit_button.pack()

    def reset_game(self):
        # Resetting game state
        self.board = ['' for _ in range(9)]
        self.player = "X"
        self.status_label.config(text="Player X's turn")
        self.mode = self.mode_var.get()
        for button in self.buttons:
            button.config(text="", state=tk.NORMAL)

    def on_button_click(self, index):
        # Handling button click event
        if self.board[index] == "":
            self.board[index] = self.player
            self.buttons[index].config(text=self.player)
            if self.check_winner(self.player):
                self.update_score(self.player)
                self.show_winner(self.player)
            elif '' not in self.board:
                self.update_score("Draw")
                self.show_winner("Draw")
            else:
                self.player = "O" if self.player == "X" else "X"
                self.status_label.config(text=f"Player {self.player}'s turn")
                if self.mode == "pvai" and self.player == "O":
                    self.master.after(500, self.ai_move)

    def ai_move(self):
        # Making AI move
        best_score = -float('inf')
        best_move = None
        for i in range(9):
            if self.board[i] == "":
                self.board[i] = "O"
                score = self.minimax(self.board, 0, False)
                self.board[i] = ""
                if score > best_score:
                    best_score = score
                    best_move = i
        if best_move is not None:
            self.on_button_click(best_move)

    def minimax(self, board, depth, is_maximizing):
        # Minimax algorithm for AI
        if self.check_winner("O"):
            return 1
        elif self.check_winner("X"):
            return -1
        elif '' not in board:
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for i in range(9):
                if board[i] == "":
                    board[i] = "O"
                    score = self.minimax(board, depth + 1, False)
                    board[i] = ""
                    best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for i in range(9):
                if board[i] == "":
                    board[i] = "X"
                    score = self.minimax(board, depth + 1, True)
                    board[i] = ""
                    best_score = min(score, best_score)
            return best_score

    def check_winner(self, player):
        # Checking if the given player has won
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertical
            [0, 4, 8], [2, 4, 6]              # Diagonal
        ]
        for combo in winning_combinations:
            if self.board[combo[0]] == self.board[combo[1]] == self.board[combo[2]] == player:
                return True
        return False

    def update_score(self, winner):
        # Updating the score based on the winner
        if winner == "X":
            self.x_score += 1
        elif winner == "O":
            self.o_score += 1
        else:
            self.draw_score += 1
        self.score_label.config(text=f"X: {self.x_score} | O: {self.o_score} | Draw: {self.draw_score}")

    def show_winner(self, winner):
        # Displaying the winner and resetting the game
        if winner == "Draw":
            messagebox.showinfo("Tic-Tac-Toe", "It's a draw!")
        else:
            messagebox.showinfo("Tic-Tac-Toe", f"Player {winner} wins!")
        self.reset_game()

if __name__ == "__main__":
    root = tk.Tk()
    game = TicTacToe(root)
    root.mainloop()
